In [1]:
from pathlib import Path

import catboost as cb
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTENC
from imblearn.under_sampling import RandomUnderSampler

In [2]:
data_path = Path("../data/")
seed = 42

In [3]:
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    n0, n1 = np.bincount(y_true.astype(int))
    w0 = 1 / (n0 / len(y_true))
    w1 = 1 / (n1 / len(y_true))

    l0 = -w0 / n0 * np.sum(np.where(y_true == 0, 1, 0) * np.log(1 - y_pred))
    l1 = -w1 / n1 * np.sum(np.where(y_true != 0, 1, 0) * np.log(y_pred))

    return (l0 + l1) / (w0 + w1)


In [4]:
# read data
train_df = pd.read_csv(data_path / "train.csv")
test_df = pd.read_csv(data_path / "test.csv")
greeks_df = pd.read_csv(data_path / "greeks.csv")

In [5]:
# some columns have trailing spaces
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
feature_cols = train_df.columns.tolist()[1:-1]

In [6]:
train_df, val_df = train_test_split(
    train_df, test_size=0.2, stratify=train_df["Class"], random_state=42
)
greeks_df = greeks_df.loc[train_df.index]

train_df = train_df.reset_index(drop=True)
greeks_df = greeks_df.reset_index(drop=True)

In [7]:
train_df.drop(columns=["Id"], inplace=True)

In [8]:
# Oversample
# over_sampler = RandomOverSampler(random_state=seed)
# train_df.fillna(train_df.mean(), numeric_only=True, inplace=True)
# over_sampler = SMOTENC(
#     categorical_features=[train_df.columns.tolist().index("EJ")], random_state=seed
# )
# train_df, _ = over_sampler.fit_resample(train_df, train_df["Class"])
under_sampler = RandomUnderSampler(random_state=seed)
train_df, _ = under_sampler.fit_resample(train_df, train_df["Class"])

In [9]:
# training
oof = np.zeros(len(train_df))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
test_probs = []
val_preds = []

In [10]:
fold = 1
for train_idx, val_idx in skf.split(train_df, train_df["Class"]):
    print(f"Fold-{fold}".center(110, "-"))
    fold += 1

    X_train, y_train = (
        train_df.loc[train_idx, feature_cols],
        train_df.loc[train_idx, "Class"],
    )

    X_val, y_val = (
        train_df.loc[val_idx, feature_cols],
        train_df.loc[val_idx, "Class"],
    )

    params = {
        "iterations": 10000,
        "early_stopping_rounds": 1000,
        # "auto_class_weights": "Balanced",
        # "scale_pos_weight": 407/86,
        # "scale_pos_weight": 10,
        "use_best_model": True,
        "random_seed": seed,
    }

    model = cb.CatBoostClassifier(**params)
    model.fit(
        X_train, y_train, eval_set=[(X_val, y_val)], cat_features=["EJ"], verbose=1000
    )

    # make oof preds
    oof[val_idx] = model.predict_proba(X_val)[:, 1]

    # make val preds
    val_preds.append(model.predict_proba(val_df[feature_cols])[:, 1])

    # make test predictions
    test_preds = model.predict_proba(test_df.iloc[:, 1:])
    test_probs.append(test_preds)

print("-" * 110)
print(f"OOF score: {balanced_log_loss(train_df['Class'], oof):.4f}")
print(f"CV score: {balanced_log_loss(val_df['Class'], np.mean(val_preds, axis=0)):.4f}")


----------------------------------------------------Fold-1----------------------------------------------------
Learning rate set to 0.007123
0:	learn: 0.6866049	test: 0.6908571	best: 0.6908571 (0)	total: 56.3ms	remaining: 9m 22s
1000:	learn: 0.0217943	test: 0.4727316	best: 0.4518241 (440)	total: 1.27s	remaining: 11.4s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.4518240599
bestIteration = 440

Shrink model to first 441 iterations.
----------------------------------------------------Fold-2----------------------------------------------------
Learning rate set to 0.007123
0:	learn: 0.6892464	test: 0.6911358	best: 0.6911358 (0)	total: 4.96ms	remaining: 49.6s
1000:	learn: 0.0268274	test: 0.2215296	best: 0.2213219 (991)	total: 1.17s	remaining: 10.5s
2000:	learn: 0.0073979	test: 0.2243687	best: 0.2119683 (1370)	total: 2.36s	remaining: 9.43s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.2119682551
bestIteration = 1370

Shrink model to first 1371

In [11]:
# Random undersampler
# OOF score: 0.3125
# CV score: 0.2881

# SMOTENC oversampler
# OOF score: 0.0754
# CV score: 0.4329

# Random oversampler
# OOF score: 0.0529
# CV score: 0.4661

# scale_pos_weight = 10
# OOF score: 0.2841
# CV score: 0.3451

# scale_pos_weight = 407/86
# OOF score: 0.3579
# CV score: 0.4470

# auto weight
# OOF score: 0.3601
# CV score: 0.4517

In [12]:
# generate a submission file
test_probs = np.mean(test_probs, axis=0)
sub_df = pd.DataFrame(
    {"Id": test_df.Id, "Class_0": test_probs[:, 0], "Class_1": test_probs[:, 1]}
)
sub_df.to_csv("submission.csv", index=False)